In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
report_dir = "../code/report/"

# Solution to the Assignment 1 of FHPC

## Part 1: theoretical model

### Naive algerithm: best $P$

First I would like to notice the various terms of $T(P,N)$:
 - $T_{\text{read}}$: time to read the numbers;
 - $(P-1)T_{\text{comm.}}$: time to communicate to $P-1$ slaves;
 - $\left(\left\lceil\frac{N}{P}\right\rceil -1\right)T_{\text{comp.}}$: time for each core (in the worst case) to compute the sum;
 - $(P-1)T_{\text{comm.}}$: time to collect the result;
 - $(P-1)T_{\text{comp.}}$: time to sum up all the partial result.
 All in all we have:
 $$
 T(P,N) = T_{\text{read}} + 2(P-1)T_{\text{comm.}} + \left(\left\lceil\frac{N}{P}\right\rceil+P-2\right)T_{\text{comp.}}\;.
 $$
 We notice a little discrepancy with formula given in the assignment. In the following we will use:
    | Time                  | Value ($\mathrm{s}$) |
    |-----------------------|---------------------:|
    | $T_{\text{read}}$     | $10^{-4}$            |
    | $T_{\text{comm.}}$    | $10^{-6}$            |
    | $T_{\text{comp.}}$    | $2\cdot 10^{-9}$     |

In [2]:
# Here we generate times for various N and P

t_read = 1e-4
t_comm = 1e-6
t_comp = 2e-9

def time(p, n):
    return t_read + 2*(p-1)*t_comm + (np.ceil(n/p)+p-2)*t_comp

p_points = np.arange(1, 1e5+1, dtype=int)
n_points = np.array([b * 10 ** e for e in [4,5,6,7,8] for b in [1,2,5]], dtype=int)

times = time(*np.meshgrid(p_points, n_points)).T

In [3]:
# Here the data are shown (just a few bit)

data = pd.DataFrame(index=p_points, data=times, columns=n_points)
data.columns.name = "N"
data.index.name = "P"

data.head().style.format("{:.2e}")

N,10000,20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000
P,,,,,,,,,,,,,,,
1,1.20e-04,1.40e-04,2.00e-04,3.00e-04,5.00e-04,1.10e-03,2.10e-03,4.10e-03,1.01e-02,2.01e-02,4.01e-02,1.00e-01,2.00e-01,4.00e-01,1.00e+00
2,1.12e-04,1.22e-04,1.52e-04,2.02e-04,3.02e-04,6.02e-04,1.10e-03,2.10e-03,5.10e-03,1.01e-02,2.01e-02,5.01e-02,1.00e-01,2.00e-01,5.00e-01
3,1.11e-04,1.17e-04,1.37e-04,1.71e-04,2.37e-04,4.37e-04,7.71e-04,1.44e-03,3.44e-03,6.77e-03,1.34e-02,3.34e-02,6.68e-02,1.33e-01,3.33e-01
4,1.11e-04,1.16e-04,1.31e-04,1.56e-04,2.06e-04,3.56e-04,6.06e-04,1.11e-03,2.61e-03,5.11e-03,1.01e-02,2.51e-02,5.01e-02,1.00e-01,2.50e-01
5,1.12e-04,1.16e-04,1.28e-04,1.48e-04,1.88e-04,3.08e-04,5.08e-04,9.08e-04,2.11e-03,4.11e-03,8.11e-03,2.01e-02,4.01e-02,8.01e-02,2.00e-01


We can seek for the minimum numerically or analitically: for a given $N$ the function $T(P,N)$ is minimized for $P\sim \sqrt{N}\cdot\sqrt{\frac{T_{\text{comp.}}}{2T_{\text{comm.}}+T_{\text{comp.}}}}$.

In [4]:
# Here we plot some value in a log-log scale

get_exp = lambda x: int(np.floor(np.log10(x)))
get_man = lambda x: x / (10 ** get_exp(x))

n_legend = [r"$N={:g}\cdot 10^{{{:d}}}$".format(get_man(n),get_exp(n)) for n in n_points ] 
fig, ax = plt.subplots()
ax.loglog(p_points, times)
ax.set_ylim(6e-5, 20e-2)
ax.set_xlabel("P")
ax.set_ylabel("time [s]")
#lgd = ax.legend(n_legend, loc='center right', bbox_to_anchor=(1.25, 0.5), prop={'size':8})
#fig.set_size_inches(w=6, h=4.7)
lgd = ax.legend(n_legend, loc='center right', ncol=2 ,bbox_to_anchor=(1.75, 0.5), prop={'size':8})
fig.set_size_inches(w=4, h=2.7)
fig.savefig(report_dir + 'th_mod_1.pgf', bbox_extra_artists=(lgd,), bbox_inches='tight')

In [5]:
# We show that numeric minimization actually coicides with analytical one

res = data.apply(['idxmin', 'min']).transpose()
res.index.name = "N"
res = res.rename(columns={"idxmin": "Pmin"})
idxmin_a = pd.Series(res.index, index=res.index).apply(lambda x: np.round(np.sqrt(x)*np.sqrt(t_comp/(2*t_comm ))))
res.insert(1, "Pmin_a", idxmin_a)
res = res.astype({"Pmin": int, "Pmin_a": int, "min": float})

res.style.format({"min": "{:.2e}"})

,Pmin,Pmin_a,min
N,,,
10000,3,3,1.11e-04
20000,4,4,1.16e-04
50000,7,7,1.26e-04
100000,10,10,1.38e-04
200000,14,14,1.55e-04
500000,22,22,1.87e-04
1000000,32,32,2.25e-04
2000000,45,45,2.77e-04
5000000,71,71,3.81e-04


In [6]:
fig, ax = plt.subplots()
ax.loglog(res['Pmin'], 'ro-')
ax.loglog(res['Pmin_a'])
ax.set_xlabel(r"$N$")
ax.set_ylabel(r"$P_{\mathrm{min.}}$")
ax.legend(["numerical", "analytical"])
fig.set_size_inches(w=4, h=2.9)
plt.savefig(report_dir + 'th_mod_2.pgf')

### Improvement of the algorithm

Perhaps it is possible to save some communication time. The idea are basically two:
 - Reduce sum (binary tree)
 
 All in all the new time is
 $$
 \tilde{T}(P,N) = T_{\text{read}} + 2\left\lceil \log_2(P)\right\rceil T_{\text{comm.}} + \left(\left\lceil\frac{N}{P}\right\rceil+\left\lceil \log_2(P)\right\rceil -1\right)T_{\text{comp.}}\;.
 $$

In [7]:
# With the new time

def time_improved(p, n):
    return t_read + 2*np.ceil(np.log2(p))*t_comm + (np.ceil(n/p)+np.ceil(np.log2(p))-1)*t_comp

p_points_improved = np.arange(1, 1e6, dtype=int)

times_improved = time_improved(*np.meshgrid(p_points_improved, n_points)).T

In [8]:
# Here the data are shown (just a few bit)

data_improved = pd.DataFrame(index=p_points_improved, data=times_improved, columns=n_points)
data_improved.columns.name = "N"
data_improved.index.name = "P"

data_improved.head().style.format("{:.2e}")

N,10000,20000,50000,100000,200000,500000,1000000,2000000,5000000,10000000,20000000,50000000,100000000,200000000,500000000
P,,,,,,,,,,,,,,,
1,1.20e-04,1.40e-04,2.00e-04,3.00e-04,5.00e-04,1.10e-03,2.10e-03,4.10e-03,1.01e-02,2.01e-02,4.01e-02,1.00e-01,2.00e-01,4.00e-01,1.00e+00
2,1.12e-04,1.22e-04,1.52e-04,2.02e-04,3.02e-04,6.02e-04,1.10e-03,2.10e-03,5.10e-03,1.01e-02,2.01e-02,5.01e-02,1.00e-01,2.00e-01,5.00e-01
3,1.11e-04,1.17e-04,1.37e-04,1.71e-04,2.37e-04,4.37e-04,7.71e-04,1.44e-03,3.44e-03,6.77e-03,1.34e-02,3.34e-02,6.68e-02,1.33e-01,3.33e-01
4,1.09e-04,1.14e-04,1.29e-04,1.54e-04,2.04e-04,3.54e-04,6.04e-04,1.10e-03,2.60e-03,5.10e-03,1.01e-02,2.51e-02,5.01e-02,1.00e-01,2.50e-01
5,1.10e-04,1.14e-04,1.26e-04,1.46e-04,1.86e-04,3.06e-04,5.06e-04,9.06e-04,2.11e-03,4.11e-03,8.11e-03,2.01e-02,4.01e-02,8.01e-02,2.00e-01


We can seek for the minimum numerically or analitically: for a given $N$ the function $T(P,N)$ is minimized for $P\sim N\frac{T_{\text{comp.}}}{2T_{\text{comm.}}+T_{\text{comp.}}}\log 2$.

In [9]:
get_exp = lambda x: int(np.floor(np.log10(x)))
get_man = lambda x: x / (10 ** get_exp(x))

n_points_new = n_points[::4]
p_points_improved_new = np.arange(1, 1e5+1, dtype=int)
times_improved_new = time_improved(*np.meshgrid(p_points_improved_new, n_points_new)).T
times_new = time(*np.meshgrid(p_points_improved_new, n_points_new)).T

n_legend = [r"$N={:g}\cdot 10^{{{:d}}}$, imp.".format(get_man(n),get_exp(n)) for n in n_points_new ] 
n_legend += [r"$N={:g}\cdot 10^{{{:d}}}$, nv.".format(get_man(n),get_exp(n)) for n in n_points_new ]
fig, ax = plt.subplots()
ax.loglog(p_points_improved_new, times_improved_new)
ax.loglog(p_points_improved_new, times_new, ':')
ax.set_ylim(6e-5, 20e-2)
ax.set_xlabel("P")
ax.set_ylabel("time [s]")
#lgd = ax.legend(n_legend, loc='center right', bbox_to_anchor=(1.38, 0.5), prop={'size':8})
#fig.set_size_inches(w=5.6, h=4.7)
lgd = ax.legend(n_legend, loc='center right', ncol=2 ,bbox_to_anchor=(1.92, 0.5), prop={'size':8})
fig.set_size_inches(w=4, h=2.7)
fig.savefig(report_dir + 'th_mod_5.pgf', bbox_extra_artists=(lgd,), bbox_inches='tight')

In [10]:
# We show that numeric minimization actually coicides with analytical one

res_improved = data_improved.apply(['idxmin', 'min']).transpose()
res_improved.index.name = "N"
res_improved = res_improved.rename(columns={"idxmin": "Pmin"})
idxmin_a = pd.Series(res_improved.index, index=res_improved.index).apply(lambda x: np.round((t_comp/(2*t_comm+t_comp))*x*np.log(2)))
res_improved.insert(1, "Pmin_a", idxmin_a)
res_improved = res_improved.astype({"Pmin": int, "Pmin_a": int, "min": float})

res_improved.style.format({"min": "{:.2e}"})

,Pmin,Pmin_a,min
N,,,
10000,8,7,1.09e-04
20000,16,14,1.11e-04
50000,32,35,1.13e-04
100000,64,69,1.15e-04
200000,128,138,1.17e-04
500000,256,346,1.20e-04
1000000,512,692,1.22e-04
2000000,1024,1385,1.24e-04
5000000,4096,3462,1.26e-04


In [11]:
fig, ax = plt.subplots()
ax.loglog(res['Pmin'], 'ro-', c='red')
ax.loglog(res_improved['Pmin'], 'ro-', c='blue')
ax.loglog(res['Pmin_a'], c='brown')
ax.loglog(res_improved['Pmin_a'], c='green')
ax.set_xlabel(r"$N$")
ax.set_ylabel(r"$P_{\mathrm{min.}}$")
lgd=ax.legend(["numerical naive", "numerical improved", "analytical naive", "analytical improved"], bbox_to_anchor=(1, 1))
fig.set_size_inches(w=4, h=2.7)
plt.savefig(report_dir + 'th_mod_4.pgf',  bbox_extra_artists=(lgd,), bbox_inches='tight')

(the discrepancies between analytical al numerical minimum point are due to the effect of upper integer part that in this case is more important)

### Comparison: Naive vs Improved Model

In [12]:
res_comp = pd.concat([res[["Pmin", "min"]].add_suffix("_naive"), res_improved[["Pmin", "min"]].add_suffix("_improved")], axis=1)
res_comp = res_comp[["Pmin_naive", "Pmin_improved", "min_naive", "min_improved"]]
res_comp.index.name = "N"
res_comp["delta"] = res_comp["min_naive"] - res_comp["min_improved"]
res_comp = res_comp.astype({"Pmin_naive": int, "Pmin_improved": int, "min_naive": float, "min_improved": float, "delta": float})

res_comp.style.format({"min_naive": "{:.2e}", "min_improved": "{:.2e}", "delta": "{:.2e}"})

,Pmin_naive,Pmin_improved,min_naive,min_improved,delta
N,,,,,
10000,3,8,1.11e-04,1.09e-04,2.17e-06
20000,4,16,1.16e-04,1.11e-04,5.50e-06
50000,7,32,1.26e-04,1.13e-04,1.32e-05
100000,10,64,1.38e-04,1.15e-04,2.29e-05
200000,14,128,1.55e-04,1.17e-04,3.75e-05
500000,22,256,1.87e-04,1.20e-04,6.76e-05
1000000,32,512,2.25e-04,1.22e-04,1.03e-04
2000000,45,1024,2.77e-04,1.24e-04,1.53e-04
5000000,71,4096,3.81e-04,1.26e-04,2.55e-04


In [13]:
res_comp.to_csv("performance-model.csv", columns=["Pmin_naive", "Pmin_improved"], index=True, header=["best P for naive algorithm", "best P for enhanced algorithm"])

## Scaling Analysis

Let us do some analytical reasoning: the speedup $S(P,N)=\frac{T(1,N)}{T(P,N)}$, can be expanded, for small $P$ (i.e. the perfect speedup limit) as
$$
S(P,N) = a_1 P - a_2 P^2 + \mathcal{O}(P^3)\;;
$$
where
$$
a_1 = \frac{T_{\text{read}}+(N-1)T_{\text{comp.}}}{N T_{\text{comp.}}}\;,\qquad a_2 = \frac{(T_{\text{read}}-2T_{\text{comm.}}-2T_{\text{comp.}})(T_{\text{read}}+(N-1)T_{\text{comp.}})}{N^2 T^2_{\text{comp.}}}\;.
$$
We notice that for our values of $T_{\text{read}}$, $T_{\text{comm.}}$ and $T_{\text{comp.}}$, we have $a_1 > 0$ and $a_2 > 0$. In order to clarify what we mean by "small $P$" we can consider the regime when $a_1P \gg a_2P^2$; this is equivalent to
$$
N\gg\frac{T_{\text{read}}-2T_{\text{comm.}}-2T_{\text{comp.}}}{T_{\text{comp.}}}P\;,
$$
which, at our values of times reads $N\gg 4.9\cdot 10^5 P$. In particular, to observe perfect speedup regime we need at least few cores, so we must have $n\gg 10^6$; then if we want to have a linear regime up to $P=P_{\text{max.}}$ we need to have
$$
N\gg 4.0\cdot 10^5 P_{\text{max.}}\;.
$$

In [14]:
def speedup(p, n):
    return time(1, n) / time(p, n)

p = p_points

n_points_new = np.array([b * 10 ** e for e in [4,5,6,7,8,9,10,11] for b in [1,2,5]], dtype=int)
scales = speedup(*np.meshgrid(p, n_points_new)).T

get_exp = lambda x: int(np.floor(np.log10(x)))
get_man = lambda x: x / (10 ** get_exp(x))

n_legend = [r"$N={:g}\cdot 10^{{{:d}}}$".format(get_man(n),get_exp(n)) for n in n_points_new ]


fig, ax = plt.subplots()
ax.loglog(p, scales)
ax.set_xlabel("P")
ax.set_ylabel("spreedup")
#lgd = ax.legend(n_legend, loc='center right', bbox_to_anchor=(1.25, 0.5), prop={'size':8})
#fig.set_size_inches(w=6, h=4.7)
lgd = ax.legend(n_legend, loc='center right', ncol=2 ,bbox_to_anchor=(1.75, 0.5), prop={'size':8})
fig.set_size_inches(w=4, h=2.7)
fig.savefig(report_dir + 'th_mod_3.pgf', bbox_extra_artists=(lgd,), bbox_inches='tight')